In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import os
import nltk
import math
import string
import re
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Input,Reshape,Conv2D,MaxPool2D,Concatenate,Flatten
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import LinearSVC


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix,precision_score,recall_score
from sklearn.metrics import confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Model

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras import regularizers
# from tensorflow.keras.layers import *

In [2]:
df= pd.read_csv('C:/Users/HP/Downloads/bert_preprocessed_train.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
df.head(3)
# df['label'].unique()

,label,tweet,id,cleaned_text
0,1,"""@KeyshawnSwag: Lmfao this cat started beating...",0,keyshawnswag lmfao this cat started beating th...
1,1,RT @digaveliavelife: Lol I be eatin da shit ou...,1,rt digaveliavelife lol i be eatin da shit outt...
2,1,RT @QueenReenie_: How bitch how? &#8220;@_Vont...,2,rt queenreenie how bitch how 8220vontethekidd ...


In [3]:
test = pd.read_csv("C:/Users/HP/Downloads/bert_preprocessed_test.csv") #,encoding='unicode_escape'

In [4]:
# Renaming the columns 
df.rename(columns = {'tweet':'original_tweet'}, inplace = True)
df.rename(columns = {'cleaned_text':'tweet'}, inplace = True)

test.rename(columns = {'tweet':'original_tweet'}, inplace = True)
test.rename(columns = {'cleaned_text':'tweet'}, inplace = True)

# df.drop('Unnamed: 0', inplace=True, axis=1)
test.drop('Unnamed: 0', inplace=True, axis=1)

In [5]:
df.head()

,label,original_tweet,id,tweet
0,1,"""@KeyshawnSwag: Lmfao this cat started beating...",0,keyshawnswag lmfao this cat started beating th...
1,1,RT @digaveliavelife: Lol I be eatin da shit ou...,1,rt digaveliavelife lol i be eatin da shit outt...
2,1,RT @QueenReenie_: How bitch how? &#8220;@_Vont...,2,rt queenreenie how bitch how 8220vontethekidd ...
3,2,Floppy bird chalmers over here taking after hi...,3,floppy bird chalmers over here taking after hi...
4,1,"""Boolin' in the bando wit a few bitches""",4,boolin in the bando wit a few bitches


In [6]:
test.head()

,original_tweet,id,tweet
0,RT @DefendWallSt: Tell me how you really feel....,0,rt defendwallst tell me how you really feel rt...
1,&#8220;@spannerhead29: @VertrellCarter #faggot...,1,8220spannerhead29 vertrellcarter faggot8221 re...
2,Ross text me back you twat,2,ross text me back you twat
3,Your name follows the word cunt in the diction...,3,your name follows the word cunt in the diction...
4,@brendacoyt_3 : ahah chill my bitch look bette...,4,brendacoyt3 ahah chill my bitch look better 1...


In [21]:
# def clean_str(in_str):
#     in_str = str(in_str)
#     # replace urls with 'url'
#     in_str = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", in_str)
#     in_str = re.sub(r'([^\s\w]|_)+', '', in_str)
#     return in_str.strip().lower()


# df['tweet'] = df['tweet'].apply(clean_str)
# test['tweet'] = test['tweet'].apply(clean_str)

In [22]:
df.label.value_counts()

1    15398
2     3291
0     1137
Name: label, dtype: int64

In [23]:
df_0 = df[df['label'] == 0].sample(frac=1)
df_1 = df[df['label'] == 1].sample(frac=1)
df_2 = df[df['label'] == 2].sample(frac=1)
sample_size = 1137
data = pd.concat([df_0.head(sample_size), df_1.head(sample_size), df_2.head(sample_size)]).sample(frac=1)

In [13]:
# data['l'] = data['tweet'].apply(lambda x: len(str(x).split(' ')))
# print("mean length of sentence: " + str(data.l.mean()))
# print("max length of sentence: " + str(data.l.max()))
# print("std dev length of sentence: " + str(data.l.std()))

### Logistic Regression with CountVectorizer

In [11]:
review = data['tweet'].values
label = data['label'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.33, random_state=42)

review_vectorizer = CountVectorizer()
review_vectorizer.fit(review_train)
Xlr_train = review_vectorizer.transform(review_train)
Xlr_test  = review_vectorizer.transform(review_test)

LRmodel = LogisticRegression()
LRmodel.fit(Xlr_train, label_train)
score = LRmodel.score(Xlr_test, label_test)
print("Accuracy:", score)

Accuracy: 0.7726465364120781


### Logistic Regression with Tf-idf

In [12]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.tweet).toarray()
labels = df.label

review_train, review_test, label_train, label_test = train_test_split(features, labels, test_size=0.33, random_state=42)

LRmodel = LogisticRegression()
LRmodel.fit(review_train, label_train)

train_score = LRmodel.score(review_train, label_train)
test_score = LRmodel.score(review_test, label_test)

print("Training Accuracy = ", train_score)
print("Test score = ", test_score)
# submission=test
# submission['label']=pred
# submission.to_csv("submission.csv", index = False)
# submission.to_csv("D:/res5.csv")

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training Accuracy =  0.931792516750734
Test score =  0.8893473941616995


In [13]:
# Generating submission file for the above cell
LRmodel = LogisticRegression()
LRmodel.fit(features, labels)

test_features = tfidf.transform(test.tweet).toarray()
pred = LRmodel.predict(test_features)

submission=test
submission['label']=pred
submission.to_csv("LR_TDFIF_CleanedData_submission.csv", index = False)
submission.to_csv("LR_CleanData.csv")

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Multinomial Naive Bayes

In [15]:
nb_tfidf = MultinomialNB()
nb_tfidf.fit(review_train, label_train)  #model
y_predict = nb_tfidf.predict(review_test)
print(classification_report(label_test,y_predict))

              precision    recall  f1-score   support

           0       0.60      0.01      0.02       366
           1       0.84      0.99      0.91      5083
           2       0.91      0.44      0.59      1094

    accuracy                           0.84      6543
   macro avg       0.78      0.48      0.51      6543
weighted avg       0.84      0.84      0.81      6543



In [16]:
## Best parameters for NB

In [17]:
# With tdfif vectorizer
params = {'alpha': [0.01,0.1,0.5,1,10],
         }

multinomial_nb_grid = GridSearchCV(MultinomialNB(), param_grid=params, n_jobs=-1, cv=5, verbose=5)
multinomial_nb_grid.fit(review_train, label_train)
y_predict = multinomial_nb_grid.predict(review_test)
print('Best Accuracy Through Grid Search : %.3f'%multinomial_nb_grid.best_score_)
print('Best Parameters : ',multinomial_nb_grid.best_params_)
print(classification_report(label_test,y_predict))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Accuracy Through Grid Search : 0.860
Best Parameters :  {'alpha': 0.1}
              precision    recall  f1-score   support

           0       0.53      0.11      0.18       366
           1       0.87      0.97      0.92      5083
           2       0.84      0.60      0.70      1094

    accuracy                           0.86      6543
   macro avg       0.75      0.56      0.60      6543
weighted avg       0.85      0.86      0.84      6543



### Decision Tree

In [19]:
# With tdfif vectorizer
nb_tfidf = DecisionTreeClassifier(criterion="entropy")
nb_tfidf.fit(review_train, label_train)  #model
y_predict = nb_tfidf.predict(review_test)
print(classification_report(label_test,y_predict))

              precision    recall  f1-score   support

           0       0.32      0.27      0.29       366
           1       0.93      0.92      0.92      5083
           2       0.78      0.83      0.80      1094

    accuracy                           0.87      6543
   macro avg       0.67      0.67      0.67      6543
weighted avg       0.87      0.87      0.87      6543



 ### Support Vector Classification

In [21]:
# I am not running this
nb_tfidf = svm.SVC(kernel='rbf', C=1,gamma='auto')
nb_tfidf.fit(review_train, label_train)  #modelb
y_predict = nb_tfidf.predict(review_test)
print(classification_report(label_test,y_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       366
           1       0.78      1.00      0.87      5083
           2       0.00      0.00      0.00      1094

    accuracy                           0.78      6543
   macro avg       0.26      0.33      0.29      6543
weighted avg       0.60      0.78      0.68      6543



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Support Vector Classification with tf-idf

In [23]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.tweet).toarray()
labels = df.label

review_train, review_test, label_train, label_test = train_test_split(features, labels, test_size=0.33, random_state=42)

svc_model = LinearSVC()
svc_model.fit(review_train, label_train)

train_score = svc_model.score(review_train, label_train)
test_score = svc_model.score(review_test, label_test)

print("Training Accuracy = ", train_score)
print("Test score = ", test_score)

# submission=test
# submission['label']=pred
# submission.drop(['', 'Phrase'], axis=1, inplace=True)
# submission.to_csv("submission.csv", index = False)
# submission.to_csv("D:/res3.csv")

Training Accuracy =  0.9634871640442672
Test score =  0.8820113097967294


In [24]:
# Generating submission file for the above cell
svc_model = LinearSVC()
svc_model.fit(features, labels)

test_features = tfidf.transform(test.tweet).toarray()
pred = svc_model.predict(test_features)

submission=test
submission['label']=pred
submission.to_csv("SVC_TDFIF_CleanedData_submission.csv", index = False)
submission.to_csv("SVC_CleanData.csv")

### Gradient Boosting

In [ ]:
gboost_m = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
train_pred = gboost_m.fit(Xlr_train, label_train).predict(Xlr_test)
print(classification_report(label_test,y_predict))


              precision    recall  f1-score   support

           0       0.67      0.68      0.68       277
           1       0.76      0.73      0.74       301
           2       0.83      0.85      0.84       275

    accuracy                           0.75       853
   macro avg       0.75      0.76      0.76       853
weighted avg       0.75      0.75      0.75       853



### Deep Learning

In [5]:
sequence_length = 32
max_features = 20000 # this is the number of words we care about

tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(df['tweet'].values)

# this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(df['tweet'].values)
x_test = tokenizer.texts_to_sequences(test['tweet'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)
x_test = pad_sequences(x_test, sequence_length)
y = pd.get_dummies(df['label']).values

# where there isn't a test set, Kim keeps back 10% of the data for testing, I'm going to do the same since we have an ok amount to play with
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print("test set size " + str(len(X_test)))

test set size 1983


### Base Model (MLP)

In [10]:
# Building the BASELINE MODEL
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
embedding_dim = 300 # Kim uses 300 here
num_filters = 100
num_words = min(max_features, len(word_index)) + 1
regularise = tf.keras.regularizers.l2(0.001)

base_model = Sequential()
base_model.add(Embedding(num_words,128,input_length=X_train.shape[1]))
base_model.add(GlobalAveragePooling1D())
base_model.add(Dense(8,activation='relu'))
base_model.add(Dense(3,activation='softmax'))
base_model.summary()
base_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history_base = base_model.fit(X_train, y_train ,epochs=20, validation_split=0.2)

from sklearn.metrics import classification_report

y_pred1 = base_model.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

# Print f1, precision, and recall scores
loss, accuracy = base_model.evaluate(X_test, y_test)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

y_test = np.argmax(y_test, axis=1)

print('Precision : ', precision_score(y_test, y_pred , average="macro"))
print('Recall : ',recall_score(y_test, y_pred , average="macro"))
print('F1-Score : ',f1_score(y_test, y_pred , average="macro"))



Found 32538 unique tokens.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 128)           2560128   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 8)                 1032      
                                                                 
 dense_1 (Dense)             (None, 3)                 27        
                                                                 
Total params: 2,561,187
Trainable params: 2,561,187
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
447/447 [==============================] - 11s 24ms/step - loss: 0.6153 - 

### Model 1: CNN - Random embeddings

In [6]:
embedding_dim = 300 # Kim uses 300 here
num_filters = 100

inputs = Input(shape=(sequence_length,), dtype='int32')

# use a random embedding for the text
embedding_layer = Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=sequence_length)(inputs)

reshape = Reshape((sequence_length, embedding_dim, 1))(embedding_layer)

# Note the relu activation which Kim specifically mentions
# He also uses an l2 constraint of 3
# Also, note that the convolution window acts on the whole 200 dimensions - that's important
conv_0 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)

# perform max pooling on each of the convoluations
maxpool_0 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2)

# concat and flatten
concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)

# do dropout and predict
dropout = Dropout(0.5)(flatten)
output = Dense(units=3, activation='softmax')(dropout)

model = Model(inputs=inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

batch_size = 32 # Kim uses 50 here, I have a slightly smaller sample size than num
history = model.fit(X_train, y_train, epochs=20, batch_size=batch_size, verbose=1, validation_split=0.1, shuffle=True)
y_pred1  = model.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

# Print f1, precision, and recall scores
loss, accuracy = model.evaluate(X_test, y_test)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

y_test = np.argmax(y_test, axis=1)

print('Precision : ', precision_score(y_test, y_pred , average="macro"))
print('Recall : ',recall_score(y_test, y_pred , average="macro"))
print('F1-Score : ',f1_score(y_test, y_pred , average="macro"))

print('Precision : ', precision_score(y_test, y_pred , average="weighted"))
print('Recall : ',recall_score(y_test, y_pred , average="weighted"))
print('F1-Score : ',f1_score(y_test, y_pred , average="weighted"))



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 300)      6000000     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 32, 300, 1)   0           ['embedding[0][0]']              
                                                                                                  
 conv2d (Conv2D)                (None, 30, 1, 100)   90100       ['reshape[0][0]']                
                                                                                              

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
# tokenizer.fit_on_texts(test['tweet'].values)
# # this takes our sentences and replaces each word with an integer
# X = tokenizer.texts_to_sequences(test['tweet'].values)
# # we then pad the sequences so they're all the same length (sequence_length)
# X = pad_sequences(X, sequence_length)

# predictions = model.predict(X)
# y_prob = model.predict(X)
# y_classes = y_prob.argmax(axis=-1) 
# test['label'] = y_classes
# test.to_csv("D:/cnn.csv")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

### Model 2: Static word2vec

In [8]:
embeddings_index = {}
f = open(os.path.join('D:/glove/', 'glove.42B.300d.txt'),encoding='utf-8') #glove.42B.300d.txt
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

embedding_dim = 300 # Kim uses 300 here
num_filters = 100
num_words = min(max_features, len(word_index)) + 1
print(num_words)

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

Found 1917494 word vectors.
Found 32538 unique tokens.
20001


In [9]:
inputs_2 = Input(shape=(sequence_length,), dtype='int32')

# note the `trainable=False`, later we will make this layer trainable
embedding_layer_2 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=False)(inputs_2)

reshape_2 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_2)

conv_0_2 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_1_2 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_2_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)

maxpool_0_2 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_2)
maxpool_1_2 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_2)
maxpool_2_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_2)

concatenated_tensor_2 = Concatenate(axis=1)([maxpool_0_2, maxpool_1_2, maxpool_2_2])
flatten_2 = Flatten()(concatenated_tensor_2)

dropout_2 = Dropout(0.5)(flatten_2)
output_2 = Dense(units=3, activation='softmax')(dropout_2)

model_2 = Model(inputs=inputs_2, outputs=output_2)
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_2.summary())

batch_size = 32
history_2 = model_2.fit(X_train, y_train, epochs=20, batch_size=batch_size, verbose=1, validation_split=0.2)
y_pred1  = model_2.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

# Print f1, precision, and recall scores
loss, accuracy = model_2.evaluate(X_test, y_test)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

y_test = np.argmax(y_test, axis=1)

print('Precision : ', precision_score(y_test, y_pred , average="macro"))
print('Recall : ',recall_score(y_test, y_pred , average="macro"))
print('F1-Score : ',f1_score(y_test, y_pred , average="macro"))


print('Precision : ', precision_score(y_test, y_pred , average="weighted"))
print('Recall : ',recall_score(y_test, y_pred , average="weighted"))
print('F1-Score : ',f1_score(y_test, y_pred , average="weighted"))

# predictions = model_2.predict(x_test)
# y_prob = model_2.predict(x_test)
# y_classes = y_prob.argmax(axis=-1) 
# test['label'] = y_classes
# test.to_csv("D:/cnn1.csv")


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 300)      6000300     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 32, 300, 1)   0           ['embedding[0][0]']              
                                                                                                  
 conv2d (Conv2D)                (None, 30, 1, 100)   90100       ['reshape[0][0]']                
                                                                                              

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# plt.plot(history_2.history['accuracy'])
# plt.plot(history_2.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

# plt.plot(history_2.history['loss'])
# plt.plot(history_2.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

### Model 3: w2v with trainable embeddings

In [9]:
inputs_3 = Input(shape=(sequence_length,), dtype='int32')
embedding_layer_3 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(inputs_3)

reshape_3 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_3)

# note the relu activation
conv_0_3 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_1_3 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_2_3 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)

maxpool_0_3 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_3)
maxpool_1_3 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_3)
maxpool_2_3 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_3)

concatenated_tensor_3 = Concatenate(axis=1)([maxpool_0_3, maxpool_1_3, maxpool_2_3])
flatten_3 = Flatten()(concatenated_tensor_3)

dropout_3 = Dropout(0.5)(flatten_3)
output_3 = Dense(units=3, activation='softmax')(dropout_3)

model_3 = Model(inputs=inputs_3, outputs=output_3)
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_3.summary())

batch_size = 32
history_3 = model_3.fit(X_train, y_train, epochs=20, batch_size=batch_size, verbose=1, validation_split=0.2)
y_pred1  = model_3.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

# Print f1, precision, and recall scores
loss, accuracy = model_3.evaluate(X_test, y_test)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

y_test = np.argmax(y_test, axis=1)

print('Precision : ', precision_score(y_test, y_pred , average="macro"))
print('Recall : ',recall_score(y_test, y_pred , average="macro"))
print('F1-Score : ',f1_score(y_test, y_pred , average="macro"))

print('Precision : ', precision_score(y_test, y_pred , average="weighted"))
print('Recall : ',recall_score(y_test, y_pred , average="weighted"))
print('F1-Score : ',f1_score(y_test, y_pred , average="weighted"))

# predictions = model_3.predict(x_test)
# y_prob = model_3.predict(x_test)
# y_classes = y_prob.argmax(axis=-1) 
# test['label'] = y_classes
# test.to_csv("D:/cnn2.csv")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 300)      6000300     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 32, 300, 1)   0           ['embedding[0][0]']              
                                                                                                  
 conv2d (Conv2D)                (None, 30, 1, 100)   90100       ['reshape[0][0]']                
                                                                                              

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# plt.plot(history_3.history['accuracy'])
# plt.plot(history_3.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

# plt.plot(history_3.history['loss'])
# plt.plot(history_3.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

In [ ]:
print("CNN random       : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))))
print("CNN static       : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))))
print("CNN trainable    : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))))
# LR = 79
# NB = 75
# DT = 73

In [10]:
### tuning the base model

In [10]:
# Removing overfitiing
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
embedding_dim = 300 # Kim uses 300 here
num_filters = 100
num_words = min(max_features, len(word_index)) + 1
regularise = tf.keras.regularizers.l2(0.001)

model_r = Sequential()
model_r.add(Embedding(num_words,128,input_length=X_train.shape[1]))
model_r.add(Dropout(0.5))
model_r.add(GlobalAveragePooling1D())
model_r.add(Dense(8,activation='relu',kernel_regularizer=regularise))
model_r.add(Dropout(0.5))
model_r.add(Dense(3,activation='softmax'))
model_r.summary()

#Compiling the model
model_r.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history_r = model_r.fit(X_train, y_train ,epochs=20, validation_split=0.2)

y_pred1  = model_r.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

# Print f1, precision, and recall scores
loss, accuracy = model_r.evaluate(X_test, y_test)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

y_test = np.argmax(y_test, axis=1)

print('Precision : ', precision_score(y_test, y_pred , average="macro"))
print('Recall : ',recall_score(y_test, y_pred , average="macro"))
print('F1-Score : ',f1_score(y_test, y_pred , average="macro"))

print('Precision : ', precision_score(y_test, y_pred , average="weighted"))
print('Recall : ',recall_score(y_test, y_pred , average="weighted"))
print('F1-Score : ',f1_score(y_test, y_pred , average="weighted"))

# predictions = model_r.predict(x_test)
# y_prob = model_r.predict(x_test)
# y_classes = y_prob.argmax(axis=-1) 
# test['label'] = y_classes
# test.to_csv("D:/cnn4.csv")

Found 32538 unique tokens.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 128)           2560128   
                                                                 
 dropout (Dropout)           (None, 32, 128)           0         
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 8)                 1032      
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 3)                 27        
                             

C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### LSTM

In [9]:
#Building the model
from keras.constraints import max_norm
regularise = tf.keras.regularizers.l2(0.001)
model1 = Sequential([
    Embedding(num_words,128,input_length=X_train.shape[1]),
    Dropout(0.5),
    LSTM(32,kernel_constraint=max_norm(3)),
    Dense(32,activation='relu',kernel_regularizer=regularise),
    Dropout(0.5),
    Dense(3,activation='softmax')
])
#Compiling the model
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#Fitting the model
history1 =  model1.fit(X_train, y_train ,epochs=20, validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=3)])
y_pred1  = model1.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

# Print f1, precision, and recall scores
loss, accuracy = model1.evaluate(X_test, y_test)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

y_test = np.argmax(y_test, axis=1)

print('Precision : ', precision_score(y_test, y_pred , average="macro"))
print('Recall : ',recall_score(y_test, y_pred , average="macro"))
print('F1-Score : ',f1_score(y_test, y_pred , average="macro"))

print('Precision : ', precision_score(y_test, y_pred , average="weighted"))
print('Recall : ',recall_score(y_test, y_pred , average="weighted"))
print('F1-Score : ',f1_score(y_test, y_pred , average="weighted"))
# predictions = model1.predict(x_test)
# y_prob = model1.predict(x_test)
# y_classes = y_prob.argmax(axis=-1) 
# test['label'] = y_classes
# test.to_csv("D:/cnn5.csv")

Epoch 1/20
447/447 [==============================] - 15s 28ms/step - loss: 0.5294 - accuracy: 0.8309 - val_loss: 0.3803 - val_accuracy: 0.8745
Epoch 2/20
447/447 [==============================] - 12s 28ms/step - loss: 0.3218 - accuracy: 0.9026 - val_loss: 0.3376 - val_accuracy: 0.8851
Epoch 3/20
447/447 [==============================] - 13s 29ms/step - loss: 0.2336 - accuracy: 0.9231 - val_loss: 0.3626 - val_accuracy: 0.8834
Epoch 4/20
447/447 [==============================] - 12s 28ms/step - loss: 0.1879 - accuracy: 0.9345 - val_loss: 0.3841 - val_accuracy: 0.8834
Epoch 5/20
62/62 [==============================] - 0s 3ms/step - loss: 0.4180 - accuracy: 0.8891
Test: accuracy = 0.889057  ;  loss = 0.418050
Precision :  0.7130921029285266
Recall :  0.6715123702608058
F1-Score :  0.6862793335191553
Precision :  0.8788099876595281
Recall :  0.8890569843671206
F1-Score :  0.8829083777899176


In [ ]:
#tuning the model

In [9]:
#Building the model
model2 = Sequential([
    Embedding(num_words,128,input_length=X_train.shape[1]),
    Bidirectional(LSTM(128,return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(32,activation='relu'),
    Dropout(0.5),
    Dense(3,activation='softmax')
])
#Compiling the model
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#Fitting the model
history2 =  model2.fit(X_train, y_train ,epochs=20, validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=3)])
y_pred1  = model2.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

# Print f1, precision, and recall scores
loss, accuracy = model2.evaluate(X_test, y_test)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

y_test = np.argmax(y_test, axis=1)

print('Precision : ', precision_score(y_test, y_pred , average="macro"))
print('Recall : ',recall_score(y_test, y_pred , average="macro"))
print('F1-Score : ',f1_score(y_test, y_pred , average="macro"))


print('Precision : ', precision_score(y_test, y_pred , average="weighted"))
print('Recall : ',recall_score(y_test, y_pred , average="weighted"))
print('F1-Score : ',f1_score(y_test, y_pred , average="weighted"))

# predictions = model2.predict(x_test)
# y_prob = model2.predict(x_test)
# y_classes = y_prob.argmax(axis=-1) 
# test['label'] = y_classes
# test.to_csv("D:/cnn6.csv")

Epoch 1/20
447/447 [==============================] - 45s 84ms/step - loss: 0.4556 - accuracy: 0.8503 - val_loss: 0.3168 - val_accuracy: 0.8902
Epoch 2/20
447/447 [==============================] - 34s 75ms/step - loss: 0.2638 - accuracy: 0.9141 - val_loss: 0.3399 - val_accuracy: 0.8829
Epoch 3/20
447/447 [==============================] - 33s 75ms/step - loss: 0.1773 - accuracy: 0.9425 - val_loss: 0.3675 - val_accuracy: 0.8762
Epoch 4/20
62/62 [==============================] - 1s 18ms/step - loss: 0.4474 - accuracy: 0.8684
Test: accuracy = 0.868381  ;  loss = 0.447356
Precision :  0.6815146141934361
Recall :  0.6440775683085872
F1-Score :  0.6560588289061624
Precision :  0.8560251067608488
Recall :  0.8683812405446294
F1-Score :  0.860889678499515
